# make grids from qr codes

Here, I combine the generated qr codes on a grid. Taking the generated images as is and printing them on 10cm x 15cm paper worked really well.

Files are saved in `{front/back}_{number}.png` format.

![This is a front example](assets/front_0.png)

![This is a back example](assets/back_0.png)

In [ ]:
import os
import re
import json

import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image

mpl.rcParams["figure.dpi"] = 500

In [ ]:
# Canvas size in centimeters
canvas_width = 15
canvas_height = 10

# Number of rows and columns of images
num_rows = 5
num_cols = 2

# Path to the folder containing PNG files
png_folder = "qr_codes"

os.makedirs("grids", exist_ok=True)

In [ ]:
# Function to extract numbers from filename
def extract_number(filename):
    match = re.search(r"\d+", filename)
    if match:
        return int(match.group())
    return float("inf")  # Return a large number for files without numbers


# Get the list of PNG files in the folder and sort them based on the numbers before the underscore
png_files = os.listdir(png_folder)
png_files = sorted(png_files, key=extract_number)

# Create a figure with the desired size and black background
fig, ax = plt.subplots(figsize=(canvas_width, canvas_height))
fig.patch.set_facecolor("black")  # Set the background color of the canvas to black

# Initialize grid index
grid_index = 0

# Plot images on canvas and save populated grids
for i in range(0, len(png_files), num_rows * num_cols):
    # Create a new grid
    ax.clear()

    # Plot images on the grid
    for j in range(num_rows * num_cols):
        if i + j < len(png_files):
            png_file = png_files[i + j]
            image = Image.open(os.path.join(png_folder, png_file))
            row = j // num_cols
            col = j % num_cols
            ax.imshow(
                image,
                extent=[
                    col * canvas_width / num_cols,
                    (col + 1) * canvas_width / num_cols,
                    (num_rows - row - 1) * canvas_height / num_rows,
                    (num_rows - row) * canvas_height / num_rows,
                ],
            )

    # Plot black lines between images for cutting
    for row in range(1, num_rows):
        ax.plot(
            [0, canvas_width],
            [row * canvas_height / num_rows, row * canvas_height / num_rows],
            color="black",
            linewidth=2,
        )
    for col in range(1, num_cols):
        ax.plot(
            [col * canvas_width / num_cols, col * canvas_width / num_cols],
            [0, canvas_height],
            color="black",
            linewidth=2,
        )

    # Add smaller margins
    margin = 0.1  # cm
    ax.set_xlim(-margin, canvas_width + margin)
    ax.set_ylim(-margin, canvas_height + margin)

    # Hide axes
    ax.axis("off")

    # Save the populated grid
    plt.savefig(
        f"grids/front_{grid_index}.png", facecolor="black", bbox_inches="tight", pad_inches=0, dpi=500
    )

    # Increment grid index
    grid_index += 1

# plot the last one

plt.show()

In [ ]:
# Function to load background colors from JSON file
def load_background_colors(filename):
    with open(filename, 'r') as f:
        colors = json.load(f)
    return colors


def get_text_colour(hex_color):
    # Convert hex color to RGB
    r = int(hex_color[1:3], 16)
    g = int(hex_color[3:5], 16)
    b = int(hex_color[5:7], 16)

    # Calculate the relative luminance
    luminance = (0.2126 * r + 0.7152 * g + 0.0722 * b) / 255

    # Choose black or white based on luminance
    return "black" if luminance > 0.5 else "white"


# Load background colors from JSON file
background_colors = load_background_colors("background_colours.json")

# Calculate total number of grids needed

# Initialize color index
color_index = 0

# Plot grids with background colors

for cur_grid in range(grid_index):
    # Create a new figure with black background
    fig, ax = plt.subplots(figsize=(canvas_width, canvas_height))
    fig.patch.set_facecolor('black')  # Set the background color of the canvas to black

    # Set the background color of the grid
    ax.set_facecolor('black')

    # Plot each cell with a different color from the background colors
    for i in range(num_rows):
        for j in range(num_cols):
            try:

                text_colour = get_text_colour(background_colors[color_index])
                ax.add_patch(plt.Rectangle((j * canvas_width / num_cols, (num_rows - i - 1) * canvas_height / num_rows),
                                        canvas_width / num_cols, canvas_height / num_rows,
                                        facecolor=background_colors[color_index], edgecolor='black'))
                
                ax.text((j + 0.5) * canvas_width / num_cols, (num_rows - i - 0.5) * canvas_height / num_rows,
                                    str(color_index+1), color=text_colour, fontsize=35, ha='center', va='center', fontfamily='RobotoMono Nerd Font Propo', fontweight='semibold')

                # Move to the next color index, wrapping around if necessary
                color_index +=1
            except IndexError:
                pass

    # Plot black lines between cells
    for i in range(1, num_rows):
        ax.axhline(i * canvas_height / num_rows, color='black', linewidth=2)
    for j in range(1, num_cols):
        ax.axvline(j * canvas_width / num_cols, color='black', linewidth=2)

    # Add margins
    margin = 0.1  # cm
    ax.set_xlim(-margin, canvas_width + margin)
    ax.set_ylim(-margin, canvas_height + margin)

    # Hide axes
    ax.axis('off')

    # Save the grid
    plt.savefig(f"grids/back_{cur_grid}.png", facecolor='black', bbox_inches='tight', pad_inches=0, dpi=500)

    # Close the figure to free memory
    plt.close(fig)